In [45]:
from sec_api import QueryApi
import json
import requests
import pandas as pd

queryAPI = QueryApi(api_key='8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9')

In [71]:
#formType:(“10-K”, “10-KT”, “10KSB”, “10KT405”, “10KSB40”, “10-K405”)
class SEC_QUERY:

    def __init__(self, form_type, ticker, size):
        self.form_type = form_type
        self.ticker = ticker
        self.size = size
        self.query = {
            "query": { 
                "query_string" : {
                    "query": f"formType:\"{self.form_type}\" AND ticker:{self.ticker}",
                }
            },
            "from": "0",
            "size": size,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
    
    def get_response(self, index:int):
        response = queryAPI.get_filings(self.query)
        return json.dumps(response["filings"][index], indent=2)
    

    def extract(self, index:int):
        query_str = self.get_response(index)
        q_dict = json.loads(query_str)
        filing_url = q_dict['linkToHtml']

        xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
        api_key = '8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9'

        final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
        response = requests.get(final_url)

        final_json = json.loads(response.text)
        return final_json

In [74]:
query1 = SEC_QUERY("10-K", "AAPL", "10")
#print(query1.get_response(0))
query1.extract(0)['StatementsOfIncome']

{'RevenueFromContractWithCustomerExcludingAssessedTax': [{'decimals': '-6',
   'unitRef': 'usd',
   'period': {'startDate': '2022-09-25', 'endDate': '2023-09-30'},
   'segment': {'dimension': 'srt:ProductOrServiceAxis',
    'value': 'us-gaap:ProductMember'},
   'value': '298085000000'},
  {'decimals': '-6',
   'unitRef': 'usd',
   'period': {'startDate': '2021-09-26', 'endDate': '2022-09-24'},
   'segment': {'dimension': 'srt:ProductOrServiceAxis',
    'value': 'us-gaap:ProductMember'},
   'value': '316199000000'},
  {'decimals': '-6',
   'unitRef': 'usd',
   'period': {'startDate': '2020-09-27', 'endDate': '2021-09-25'},
   'segment': {'dimension': 'srt:ProductOrServiceAxis',
    'value': 'us-gaap:ProductMember'},
   'value': '297392000000'},
  {'decimals': '-6',
   'unitRef': 'usd',
   'period': {'startDate': '2022-09-25', 'endDate': '2023-09-30'},
   'segment': {'dimension': 'srt:ProductOrServiceAxis',
    'value': 'us-gaap:ServiceMember'},
   'value': '85200000000'},
  {'decimals':

In [69]:
# def extract(query_obj:SEC_QUERY, index: int):
#     #convert query_object string to dict
#     query_str = query_obj.get_response(index)
#     q_dict = json.loads(query_str)
#     filing_url = q_dict['linkToHtml']

#     xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
#     api_key = '8f1d7d5c5748f2c4a8f3d9f2961a7f633dff4a408229f567f9e2031be69906c9'

#     final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
#     response = requests.get(final_url)

#     final_json = json.loads(response.text)
#     return final_json

In [70]:
extract(query1, 0)

{'CoverPage': {'DocumentType': '10-K',
  'DocumentAnnualReport': 'true',
  'DocumentPeriodEndDate': '2023-12-31',
  'CurrentFiscalYearEndDate': '--12-31',
  'DocumentTransitionReport': 'false',
  'EntityFileNumber': '001-34756',
  'EntityRegistrantName': 'Tesla, Inc.',
  'EntityIncorporationStateCountryCode': 'DE',
  'EntityTaxIdentificationNumber': '91-2197729',
  'EntityAddressAddressLine1': '1 Tesla Road',
  'EntityAddressCityOrTown': 'Austin',
  'EntityAddressStateOrProvince': 'TX',
  'EntityAddressPostalZipCode': '78725',
  'CityAreaCode': '512',
  'LocalPhoneNumber': '516-8177',
  'Security12bTitle': 'Common stock',
  'TradingSymbol': 'TSLA',
  'SecurityExchangeName': 'NASDAQ',
  'EntityWellKnownSeasonedIssuer': 'Yes',
  'EntityVoluntaryFilers': 'No',
  'EntityCurrentReportingStatus': 'Yes',
  'EntityInteractiveDataCurrent': 'Yes',
  'EntityFilerCategory': 'Large Accelerated Filer',
  'EntitySmallBusiness': 'false',
  'EntityEmergingGrowthCompany': 'false',
  'IcfrAuditorAttestat